In [1]:
import json
import os

PUBTAB_ANN = '../../pubtabnet/anns/train/'
PUBTAB_IMG = '../../pubtabnet/imgs/train/'

ANN_PATH = PUBTAB_ANN
IMAGE_PATH = PUBTAB_IMG
IMG_FORMAT = '.png'

with open(ANN_PATH[:-1] + '_outlier_trunc_filelist.json') as file:
    unallowed_list = file.read().splitlines()
with open(ANN_PATH[:-1] + '_trunc_filelist.json') as file:
    allowed_list = file.read().splitlines()

In [2]:
aux_list = []

for json_item in allowed_list:
    aux_list.append(json_item[:-5])

allowed_list = aux_list

In [3]:
aux_list = []

for json_item in unallowed_list:
    aux_list.append(json_item[:-5])

unallowed_list = aux_list

In [4]:
unallowed_list[:10]

['PMC4357343_004_00',
 'PMC2873801_006_00',
 'PMC5790457_005_00',
 'PMC4882839_006_00',
 'PMC4425873_002_00',
 'PMC5707723_004_00',
 'PMC4335538_003_00',
 'PMC3797567_004_00',
 'PMC2944093_004_00',
 'PMC3987684_005_00']

In [5]:
allowed_list[:10]

['PMC6051241_006_00',
 'PMC3398398_014_00',
 'PMC2682793_006_00',
 'PMC6082926_006_00',
 'PMC2809502_009_00',
 'PMC3890642_004_00',
 'PMC3926592_006_01',
 'PMC548940_003_00',
 'PMC4394591_006_00',
 'PMC1906827_006_00']

In [6]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from transformers import VisionEncoderDecoderConfig
import torch

image_size = [750, 750]
max_length = 4096#config.decoder.max_position_embeddings

processor = DonutProcessor.from_pretrained("../../pubtabnet/modelos/Donut_PubTables_TML_Processor")
config = VisionEncoderDecoderConfig.from_pretrained("naver-clova-ix/donut-base")

2023-10-28 19:51:00.073266: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-28 19:51:00.073290: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-28 19:51:00.073307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-28 19:51:00.716499: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
cell_types = ["<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
for i in range(2):
    for j in range(2):
        for k in range(2):
            cell_types.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            cell_types.append("<span_type=1" + str(i) + str(j) + str(k) + ">")


cell_tokens = [processor.tokenizer.convert_tokens_to_ids([cell_type])[0] for cell_type in cell_types]
row_tokens = [processor.tokenizer.convert_tokens_to_ids([row_type])[0] for row_type in ['<row>', '</s>']]

In [8]:
new_config = config.decoder

new_config.pos_counters = [cell_tokens, row_tokens]
new_config.dim_max_position_embeddings = [5000, 300, 300]
new_config.architectures = "DiMBartForCausalLM"
new_config.model_type = "dimbart"

In [9]:
from transformers import DiMBartConfig

new_config = DiMBartConfig(**new_config.__dict__)
new_config.pos_counters = new_config.pos_counters

In [15]:
config.decoder = new_config

In [16]:
config

VisionEncoderDecoderConfig {
  "_name_or_path": "naver-clova-ix/donut-base",
  "architectures": [
    "VisionEncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_cross_attention": true,
    "add_final_layer_norm": true,
    "architectures": "DiMBartForCausalLM",
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 1024,
    "decoder_attention_heads": 16,
    "decoder_ffn_dim": 4096,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 4,
    "decoder_start_token_id": null,
    "dim_max_position_embeddings": [
      5000,
      300,
      300
    ],
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_attention_heads": 16,
    "encoder_ffn_dim": 4096,
    "enc

In [17]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base", config = config, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at naver-clova-ix/donut-base were not used when initializing VisionEncoderDecoderModel: ['decoder.model.decoder.embed_positions.weight', 'decoder.model.decoder.layer_norm.weight', 'decoder.model.decoder.layer_norm.bias']
- This IS expected if you are initializing VisionEncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisionEncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at naver-clova-ix/donut-base and are newly initialized: ['decoder.model.decoder.embed_positions.embeddings.0.weight', 'decoder.model.decoder.embed_positions.embeddings.

In [18]:
model.save_pretrained('aux_model')

In [ ]:
with open("msg.json", 'w') as out:
        json.dump({'outputs': []}, out, ensure_ascii=False, indent=4)

def write_msg(msg):
    with open("msg.json", encoding="utf-8") as f:
        json_data = json.load(f)
    
    with open("msg.json", 'w') as out:
        json_data['outputs'].append(msg)
        json.dump(json_data, out, ensure_ascii=False, indent=4)

In [ ]:
def cel2token(cell):
    if cell['span_type'][10:] != '0000':
        sequence = "<" + cell['span_type'] + ">"
    else:
        sequence = ""
    
    if cell['span_type'][10:] in ['0000', '0100', '1000', '1100']:
        if cell['row_header'] and cell['col_header']:
            sequence += "<row_and_col_header>"
        elif cell['col_header']:
            sequence += "<col_header>"
        elif cell['row_header']:
            sequence += "<row_header>"
        else:
            sequence += "<cell>"
        sequence += cell['content']
        
    return sequence

def row2token(row):
    sequence = "<row>"
    for cell in row:
        sequence += cel2token(cell)
    
    return sequence


def table2token(table):
    sequence = "<table>"
    for row in table:
        sequence += row2token(row)
    
    return sequence


def json2token(json):
    sequence = ""
    if('tables' in json):
        for table in json['tables']:
            sequence += table2token(table)

    return sequence

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class DonutTableDataset(Dataset):
    def __init__(
        self,
        annotations,
        image_size,
        max_length,
        shuffle = True,
        split = "train",
        ignore_id = -100,
        prompt_end_token = None,
    ):            
        self.annotations = annotations
        
        
        self.image_size = image_size
        self.max_length = max_length
        self.split = split
        self.ignore_id = ignore_id
        
        self.resize = transforms.Compose([transforms.Resize(image_size)])
        
        
        
    def __len__(self):
        return len(self.annotations)
    
    
    def __getitem__(self, idx):
        
        file_name = self.annotations[idx]
        
        with open(ANN_PATH + file_name + ".json", encoding="utf-8") as f:
            annotation = json.load(f)
        
        image = Image.open(IMAGE_PATH + file_name + IMG_FORMAT)
        
        
        # inputs
        pixel_values = processor(image.convert("RGB"), random_padding=self.split == "train", return_tensors="pt").pixel_values.squeeze()
        pixel_values = pixel_values.squeeze()
        
        target_sequence = json2token(annotation)
        
        input_ids = processor.tokenizer(
            target_sequence,
            add_special_tokens=True,
            max_length= max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id
        
        
        encoding = dict(pixel_values=pixel_values,
                        labels=labels)
        
        return encoding

In [ ]:
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(["<table_extraction>"])[0] 


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
start_epoch = 0
avg_size = 1000

In [ ]:
import torch
from random import choices
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model.to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)


for epoch in range(start_epoch, 10):
    allowed_smpl = choices(allowed_list, k = len(unallowed_list))
    train_dataset = DonutTableDataset(unallowed_list + allowed_smpl, max_length = max_length, image_size = image_size)
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    
    
    print("Epoch:", epoch+1)
    mean_loss = 0
    mean_smpl_loss = 0 
    model.train()
    for i, batch in enumerate(tqdm(train_dataloader)):
        
        batch = {k: v.to(device) for k, v in batch.items()}
        
        pixel_values = batch["pixel_values"]
        labels = batch["labels"]
        
        
        outputs = model(pixel_values=pixel_values, labels=labels)
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        mean_loss += loss.item()   
        mean_smpl_loss += loss.item() 
        if i % avg_size == 0:
            print("Loss: ", mean_smpl_loss/avg_size)
            write_msg("batch " + str(i) +" loss: "+ str(mean_smpl_loss/avg_size))
            mean_smpl_loss = 0 
        
        if i % 10000 == 0:
            model.save_pretrained("model_epoch_checkpoint")
    
    torch.save(model, "model_epoch_checkpoint_.bin")
    print("Epoch's mean loss: ", mean_loss/len(train_dataloader))
    
    write_msg("Epoch checkpointed: " + str(epoch+1) +" \n"+
              "Epoch's mean Loss: " + str(mean_loss/len(train_dataloader)))

In [ ]:
len(train_dataset)

In [ ]:
model.save_pretrained("../../pubtabnet/model-minimal-long-epoch10")

In [ ]:
processor.save_pretrained("../../pubtabnet/Donut_PubTables_Processor")